In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
os.getenv("MLFLOW_TRACKING_URI")
os.getenv("MLFLOW_TRACKING_USERNAME")
os.getenv("MLFLOW_TRACKING_PASSWORD")

In [3]:
%pwd

'/home/mushfiq/Desktop/End-to-End-MLOPS/ds_end_to_end/research'

In [4]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
from src.ds_end_to_end.constants import *
from src.ds_end_to_end.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(config.test_data_path), #config.test_data_path,
            model_path=Path(config.model_path), #config.model_path,
            all_params=params,
            metric_file_name=Path(config.metric_file_name), #config.metric_file_name,
            target_column=schema.name,
            mlflow_uri=os.getenv("MLFLOW_TRACKING_URI"),
        )

        return model_evaluation_config

In [8]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import mlflow
import mlflow.sklearn
import joblib
from pathlib import Path
from urllib.parse import urlparse
from src.ds_end_to_end import logger

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path) # Load the trained model

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        # No need to check tracking_url_type_store if we are always using log_artifact
        # tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local JSON file
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=self.config.metric_file_name, content=scores)
            logger.info(f"Metrics saved to: {self.config.metric_file_name}")

            # Log parameters and metrics to MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            mlflow.log_artifact(local_path=str(self.config.model_path), artifact_path="model")
            logger.info(f"Model logged as artifact '{self.config.model_path.name}' to MLflow run.")

        logger.info("Metrics and model logged to MLflow successfully!")

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-22 23:18:44,950: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-22 23:18:44,955: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-22 23:18:44,964: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-22 23:18:45,830: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
[2025-07-22 23:18:45,838: INFO: 2370368008: Metrics saved to: artifacts/model_evaluation/metrics.json]
[2025-07-22 23:18:52,080: INFO: 2370368008: Model logged as artifact 'model.joblib' to MLflow run.]
🏃 View run serious-doe-242 at: https://dagshub.com/cuetian.mushfiq/End-to-End-MLOPS.mlflow/#/experiments/0/runs/abc55849fd76419daeb86843a7daae8e
🧪 View experiment at: https://dagshub.com/cuetian.mushfiq/End-to-End-MLOPS.mlflow/#/experiments/0
[2025-07-22 23:18:53,635: INFO: 2370368008: Metrics and model logged to MLflow successfully!]
